# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pandas as pd
import re
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
# Had to use table name created in previous ETL notebook
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Not sure if this is column is useful or not
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [26]:
# going to try using just message at first, but not sure if I should also be using genre
# X = df[['message']] 
X = df['message'].values
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
category_names = Y.columns

In [27]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [10]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [11]:
X.shape

(26216,)

In [12]:
Y[:3]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
Y.iloc[0].shape

(36,)

In [14]:
Y.iloc[0]

related                   1
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [15]:
Y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [19]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [17]:
# normalize (lower case, no punctuation), remove stop words, lemmatize
# NLP Pipelines lesson 21 Notebook for example
def tokenize(text):
    """Prepare text by converting to lowercase, removing punctuation and stop words, and lemmatizing"""
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

In [18]:
sample_corpus = "Here is a sample sentence ;..28 swimming hiding run running on"
tokenize(sample_corpus)

['sample', 'sentence', '28', 'swimming', 'hiding', 'run', 'running']

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
# TRYING WITHOUT MULTIOUTPUT FOR A SECOND
# Pass tokenize function into CountVectorizer
# Looks like MultiOutputClassifier takes an estimator as an argument
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    # ('clf', MultiOutputClassifier(forest))
])

# estimator: an estimator object implementing fit, score and predict_proba
# MultiOutputClassifier(estimator)

In [21]:
# to view parameters in pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f6a73410840>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
# http://scikit-learn.org/stable/modules/multiclass.html
# Do I need to use MultiLabelBinarizer to convert my ys?
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

NameError: name 'X_train' is not defined

In [76]:
print(X_train.iloc[0][0])

we are in rue Dessalines Petit goaves and it necessary to..? 


In [77]:
print(X_train.iloc[1][0])

The north, west and several other parts of the country are experiencing severe food insecurity due to erratic rains and a poor harvest in 2011.


In [78]:
X_train[:3]

,message
3121,we are in rue Dessalines Petit goaves and it n...
16039,"The north, west and several other parts of the..."
12822,Happy hurricane wishes ! http://t.co/KDwz7Qjf


In [79]:
# How is the related "2" in the below sample for id 3121? I thought everything was a 1 or a 0?
Y_train[:2]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3121,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16039,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [80]:
X.shape

(26216, 1)

In [81]:
Y.shape

(26216, 36)

In [ ]:
"""
FROM LESSON 4 ML PIPELINE #7
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    clf = RandomForestClassifier()

    # train classifier
    X_train_counts = vect.fit_transform(X_train)
    X_train_tfidf = tfidf.fit_transform(X_train_counts)
    clf.fit(X_train_tfidf, y_train)

    # predict on test data
    X_test_counts = vect.transform(X_test)
    X_test_tfidf = tfidf.transform(X_test_counts)
    y_pred = clf.predict(X_test_tfidf)

    # display results
    display_results(y_test, y_pred)
"""

In [23]:
# Hmm, probably need to think about if I can pass in Y variable like I have it right now
# Error I'm getting: Found input variables with inconsistent numbers of samples: [1, 20972]
# Example on how to possible fix it:
# https://datascience.stackexchange.com/questions/20199/train-test-split-error-found-input-variables-with-inconsistent-numbers-of-sam

# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# train pipeline
# NEED TO TOKENIZE X_TRAIN FIRST
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall on both the training set and the test set. You can use sklearn's `classification_report` function here. 

In [31]:
y_pred_train = pipeline.predict(X_train)
train_results = classification_report(Y_train, y_pred_train.values, target_names=category_names)
print(train_results)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
# test results
y_pred_test = pipeline.predict(X_test)
test_results = classification_report(Y_test, y_pred_test, target_names=category_names)
print(test_results)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 3]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)
cv_y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

In [ ]:
# classification_report

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# maybe try a SVC or a simple neural network

### 9. Export your model as a pickle file

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_pickle.html
# path: where pickled object will be stored
# df.to_pickle(path)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.